In [ ]:
#Import Required Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import xarray as xr
import salem

Input Data for Rainfall erosivity Factor and Indices are 

Hourly (IMDAA) 
and Daily (IMD and CHIRPS) rainfall DAta

ALl the input .nc data avaialble in 'imdaaa', 'imd', and 'chirps' folders

# Rainfall Erosivity Factor

Our target is to calculate long term annual average rainfall erosivity factor 

In [ ]:
# To open all the .nc files of 'imdaa' folder altogether

ncfile = ('imdaa/*.nc')
ncdat = xr.open_mfdataset(ncfile)


In [ ]:
#Import Indian Political Boubdary Shapfile and clip the files for India only ((Optional))

indiashp=gpd.read_file('Shapefiles/Indian_states.shp')
ncdat_india = ncdat.salem.subset(shape=indiashp).salem.roi(shape=indiashp)

Note: IMDAA Hourly data used for R-factor, rainfall intensity is in per hour (i=1 hour), event= 1 hour, 
I60= Intensity in 60 minutes, I30= Rainfall intensity in 30 minutes, EI60=Rainfall Intensity in 60 minutes


#Step to Calculate R-factor

1) Calculate the kinetic energy of the event (Ek)
(There are six empirical equations are there to calculate Ek) (Raj et al 2022)

2) Calculate the Rainfall Intensity for 60 minutes (EI60)
(Here EI60 was considered as EI30 as no sub-hourly data was available)

3) Calculate Rainfall erosivity factor the event (Rk)

4) Calculate Yearly rainfall erosivity factor (R-factor) for n years 
(n should be greater than 20)

For Example:

Van Dijk et al 2002

ek = 0.283*(1-0.52*exp(-0.042i));
(i= rainfall intensity)

In [ ]:
#Van Dijk et al 2002
#er = 0.283*(1-0.52*exp(-0.042i))

a1 = -0.042 * ncdat_india['APCP_sfc']
b1 = np.exp(a1)
c1 = (1 - 0.52 * b1)

ek = 0.283 * c1

I60 = ncdat_india['APCP_sfc']

EI60 = ek  * I60

Rk = EI60 * I60

In [ ]:
#sum of erosive events yearwise
Ryr = Rk.resample(time='y',label='right').sum()

#To convert chunks into array
Ryr = Ryr.compute()

In [ ]:
#Mean of yearwise R-values
xx = Ryr.mean('time')

In [ ]:
#Subsetting for India
R_ind = xx.salem.subset(shape=indiashp).salem.roi(shape=indiashp)

In [ ]:
#Plotting and Saving
R_ind.to_nc('R-factor_India.nc')
R_ind.plot()

# Rainfall Erosivity Indices

All the three datasets were used to estimate FI and MFI:

IMDAA Hourly

IMD Daily

CHIRPS Daily 



# FI Calculation

Using IMD Daily Data

In [ ]:
# Open all .nc files in imd folder altogether to process

ncfile = ('imd/*.nc')
ncdat = xr.open_mfdataset(ncfile)

Fournier Index
FI= [(p_max)^2]/P

FI = p^2 / P
where:
FI = Fourier Index (Rainfall Erosivity Index)
p = Rainfall of the wettest month in the year
P = Annual rainfall


In [ ]:
# Resample it to monthly, yearly and get max_monthly rainfall

nc_monthly=ncdat.resample(time='m').sum()

nc_month_max=nc_monthly.resample(time='y').max()
monthly_max_sq =  (nc_month_max * nc_month_max)
nc_month_max_year = monthly_max_sq.sum('time')

nc_yearly=ncdat.resample(time='y').sum()
nc_year_sum = nc_yearly.sum('time')  #Sum of yearly mean for all years


In [ ]:
# Final FI 
fidat = (nc_month_max_year/nc_year_sum)

# Save FI
fidat.to_nc('FI_IMD.nc')

# Modified Fournier Index

MFI = (1/P) * sum(p_i^2)


where:


MFI = Modified Fournier Index; 
P = Annual rainfall;
p_i = Monthly rainfall for month i;
sum is taken over all 12 months of the year


In [ ]:
# Open all .nc files in imd folder altogether to process

ncfile = ('imd/*.nc')
ncdat = xr.open_mfdataset(ncfile)

In [ ]:
# resample as monthly and yearly rainfall 

nc_month_mean=ncdat.resample(time='m').mean()
nc_year_mean=ncdat.resample(time='y').mean()

In [ ]:
def calcMFI(ncdat):
    '''Calculate Modified Fournier Indices (MFI)
    
    Parameters:
        ncdat: xarray dataset
    
    Returns:
        mfidat: A netCDF file with MFI values
    '''
    nc_month_sq = np.square(nc_month_mean).sum('time') #Square of all mean monthly values and sum of all years
    nc_year_sum = nc_year_mean.sum('time')  #Sum of yearly mean of all years
    mfidat = (nc_month_sq/nc_year_sum)
    
    return(mfidat)

In [ ]:
# Calculate and Plot MFI

#MFI
mfi = calcMFI(ncdat)
mfi['rf'].plot(levels=10, cmap = 'RdYlBu_r', cbar_kwargs={'label': 'Modified Fournier Index'})
plt.title('Modified Fournier Index [1901-2019]')

In [ ]:
# Save FI
mfi.to_nc('MFI_IMD.nc')

NOTE: YOU CAN USE THE SAME FOR IMDAA hourly and CHIRPS Daily data